In [1]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify
import seaborn as sns
import random
import numpy as np
import math
from sklearn.model_selection import train_test_split
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Activation, Flatten, Dense, Input, MaxPooling2D, Add, Reshape, concatenate, AveragePooling2D, Multiply, GlobalAveragePooling2D, UpSampling2D, MaxPool2D,Softmax
from tensorflow.keras.activations import softmax
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.optimizers import Adam
import pickle

In [2]:
def patches(img, patch_size):
    patches = patchify(img, (patch_size, patch_size, 3), step=patch_size)
    return patches

In [3]:
#Custom function to get denoised image prediction for noisy images
def prediction(img,model):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(1024,1024))
  img = img.astype("float32") / 255.0

  img_patches = patches(img,256)

  nsy=[]
  for i in range(4):
    for j in range(4):
      nsy.append(img_patches[i][j][0])
  nsy = np.array(nsy)

  pred_img = model.predict(nsy)
  pred_img = np.reshape(pred_img,(4,4,1,256,256,3))
  pred_img = unpatchify(pred_img, img.shape)
  return pred_img


#Custom function to get denoised image prediction for noisy images on quantized models using tflite
def prediction_tflite(img,model):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(1024,1024))
  img = img.astype("float32") / 255.0

  img_patches = patches(img,256)

  nsy=[]
  for i in range(4):
    for j in range(4):
      nsy.append(img_patches[i][j][0])
  nsy = np.array(nsy)
  pred=[]
  for patch in nsy:
    model.set_tensor(input_details[0]['index'], tf.expand_dims(patch,axis=0))
    model.invoke()
    tflite_model_predictions = model.get_tensor(output_details[0]['index'])
    pred.append(tflite_model_predictions)

  pred_img = np.reshape(pred,(4,4,1,256,256,3))
  pred_img = unpatchify(pred_img, img.shape)
  return pred_img

#Custom function to plot/visualize noisy, ground truth and predicted images
def visualize(sample,model):
  fig,ax = plt.subplots(len(sample),3,figsize=(30,30))
  for i in range(len(sample)):
    path = sample['Ground Truth Images'].iloc[i]
    test_img_gt = cv2.imread(path)
    test_img_gt = cv2.cvtColor(test_img_gt, cv2.COLOR_BGR2RGB)
    test_img_gt = cv2.resize(test_img_gt,(512,512))
    test_img_gt = test_img_gt.astype("float32") / 255.0
  
    path = sample['Noisy Images'].iloc[i]
    test_img_nsy = cv2.imread(path)
    pred_img = prediction(test_img_nsy,model)
    pred_img = cv2.resize(pred_img,(512,512))

    test_img_nsy = cv2.cvtColor(test_img_nsy, cv2.COLOR_BGR2RGB)
    test_img_nsy = cv2.resize(test_img_nsy,(512,512))
    test_img_nsy = test_img_nsy.astype("float32") / 255.0
    
    ax[i][0].imshow(test_img_nsy)
    ax[i][0].get_xaxis().set_visible(False)
    ax[i][0].get_yaxis().set_visible(False)
    ax[i][0].title.set_text("Noisy Image")

    ax[i][1].imshow(test_img_gt)
    ax[i][1].get_xaxis().set_visible(False)
    ax[i][1].get_yaxis().set_visible(False)
    ax[i][1].title.set_text("Ground Truth Image")

    ax[i][2].imshow(pred_img)
    ax[i][2].get_xaxis().set_visible(False)
    ax[i][2].get_yaxis().set_visible(False)
    ax[i][2].title.set_text("Predicted Image")

#Custom function that computes the psnr and ssim values for images
def psnr_and_ssim(X_test,y_test,model,model_type='Normal'):
  psnr_nsy = 0.0
  psnr_de_nsy = 0.0
  ssim_nsy = 0.0
  ssim_de_nsy = 0.0
  for i in range(len(X_test)):
    #getting the noisy images
    path = X_test.iloc[i]
    nsy = cv2.imread(path)  

    #getting the predicted images
    if model_type == 'Quantized': 
      pred = prediction_tflite(nsy,model)
    else:
      pred = prediction(nsy,model)

    #getting the ground truth images
    path = y_test.iloc[i]
    gt = cv2.imread(path)         
    gt = cv2.cvtColor(gt, cv2.COLOR_BGR2RGB)
    
    #Resizing the images
    gt = cv2.resize(gt,(1024,1024))
    nsy = cv2.resize(nsy,(1024,1024))

    #Normalizing the images
    gt = gt.astype("float32") / 255.0
    nsy = nsy.astype("float32") / 255.0

    #Computing psnr and ssim for test images
    psnr_nsy += psnr(gt,nsy)
    psnr_de_nsy += psnr(gt,pred)
    ssim_nsy += ssim(gt,nsy,multichannel=True,data_range=nsy.max() - nsy.min())
    ssim_de_nsy += ssim(gt,pred,multichannel=True,data_range=pred.max() - pred.min())

  psnr_nsy = psnr_nsy/len(X_test)
  psnr_de_nsy = psnr_de_nsy/len(X_test)
  ssim_nsy = ssim_nsy/len(X_test)
  ssim_de_nsy = ssim_de_nsy/len(X_test)
  return psnr_nsy, psnr_de_nsy,ssim_nsy,ssim_de_nsy

In [5]:
X_train_patches, y_train_patches = pickle.load(open('train_data.pkl', 'rb'))
X_test_patches,y_test_patches = pickle.load(open('test_data.pkl', 'rb'))

In [6]:
print(X_train_patches.shape)
print(y_train_patches.shape)
print(X_test_patches.shape)
print(y_test_patches.shape)

(4096, 256, 256, 3)
(4096, 256, 256, 3)
(1024, 256, 256, 3)
(1024, 256, 256, 3)


In [7]:
# Normalizing the image pixels
X_train_patches = X_train_patches.astype('float32') / 255.0
y_train_patches = y_train_patches.astype('float32') / 255.0
X_test_patches = X_test_patches.astype('float32') / 255.0
y_test_patches = y_test_patches.astype('float32') / 255.0

In [8]:
class Dataloader(tf.keras.utils.Sequence):
    def __init__(self, X, y, batch_size=1, shuffle=False):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(X))

    def __getitem__(self, i):
        batch_x = self.X[i * self.batch_size : (i + 1) * self.batch_size]
        batch_y= self.y[i * self.batch_size : (i + 1) * self.batch_size]
        return tuple((batch_x, batch_y))

    def __len__(self):
        return len(self.indexes)

    def on_epoch_end(self):
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

In [9]:
batch_size = 32
train_dataloader = Dataloader(X_train_patches,y_train_patches, batch_size, shuffle=True)
test_dataloader = Dataloader(X_test_patches,y_test_patches,batch_size, shuffle=True)

In [10]:
train_dataloader[0][0].shape

(32, 256, 256, 3)

In [6]:
tf.keras.backend.clear_session()

input = Input(shape=(256, 256, 3))

# Noise estimation subnetwork
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(input)
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = Conv2D(3, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)

# Non Blind denoising subnetwork
x = concatenate([x, input])
conv1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)

pool1 = AveragePooling2D(pool_size=(2, 2), padding='same')(conv2)
conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
conv5 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)

pool2 = AveragePooling2D(pool_size=(2, 2), padding='same')(conv5)
conv6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
conv7 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)
conv8 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
conv9 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv8)
conv10 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv9)
conv11 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv10)

upsample1 = Conv2DTranspose(128, (3, 3), strides=2, activation="relu", kernel_initializer='he_normal',padding="same")(conv11)
add1 = Add()([upsample1,conv5])
conv12 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(add1)
conv13 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv12)
conv14 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv13)

upsample2 = Conv2DTranspose(64, (3, 3), strides=2, activation="relu", kernel_initializer='he_normal',padding="same")(conv14)
add1 = Add()([upsample2, conv2])
conv15 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(add1)
conv16 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv15)

out = Conv2D(3, (1, 1), kernel_initializer='he_normal', padding='same')(conv16)
out = Add()([out, input])

CBDNet = Model(input, out)
CBDNet.compile(optimizer=tf.keras.optimizers.Adam(1e-03), loss=tf.keras.losses.MeanSquaredError())
CBDNet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 32  9248        ['conv2d[0][0]']                 
                                )                                                             

In [15]:
log_dir="logs/cbdnet"

tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,verbose=1,patience=4,min_delta=0.00001)
callback = [tensorboard,reducelr]
CBDNet.fit(train_dataloader,shuffle=True,epochs=30,validation_data= test_dataloader,callbacks=callback)

Epoch 1/30
4096/4096 [==============================] - 128s 31ms/step - loss: 67957304590336.0000 - val_loss: 0.0027 - lr: 0.0010
Epoch 2/30
4096/4096 [==============================] - 108s 26ms/step - loss: 2.7074e-05 - val_loss: 0.0027 - lr: 0.0010
Epoch 3/30
4096/4096 [==============================] - 109s 27ms/step - loss: 2.7074e-05 - val_loss: 0.0027 - lr: 0.0010
Epoch 4/30
4096/4096 [==============================] - 108s 26ms/step - loss: 2.7074e-05 - val_loss: 0.0027 - lr: 0.0010
Epoch 5/30
4089/4096 [============================>.] - ETA: 0s - loss: 2.7073e-05
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
4096/4096 [==============================] - 108s 26ms/step - loss: 2.7073e-05 - val_loss: 0.0027 - lr: 0.0010
Epoch 6/30
4096/4096 [==============================] - 108s 26ms/step - loss: 2.7069e-05 - val_loss: 0.0027 - lr: 1.0000e-04
Epoch 7/30
4096/4096 [==============================] - 108s 26ms/step - loss: 2.7055e-05 - val_loss: 0.00

In [4]:
%tensorboard --logdir "/logs/cbdnet"

UsageError: Line magic function `%tensorboard` not found.


In [17]:
CBDNet.save('models/CBDNet', save_format='tf')

INFO:tensorflow:Assets written to: models/CBDNet\assets


INFO:tensorflow:Assets written to: models/CBDNet\assets


In [4]:
CBDNet = tf.keras.models.load_model('models/CBDNet')

In [5]:
sample = pd.DataFrame({'Ground Truth Images' : ['datasets/GT3.png'], 'Noisy Images' : ['datasets/N3.png']})

In [10]:
path = sample['Ground Truth Images'].iloc[0]
test_img_gt = cv2.imread(path)
test_img_gt = cv2.cvtColor(test_img_gt, cv2.COLOR_BGR2RGB)
test_img_gt = cv2.resize(test_img_gt, (512, 512))
test_img_gt = test_img_gt.astype("float32") / 255.0

path = sample['Noisy Images'].iloc[0]
test_img_nsy = cv2.imread(path)

pred = prediction(test_img_nsy, CBDNet)
pred = cv2.resize(pred, (512, 512))

test_img_nsy= cv2.cvtColor(test_img_nsy, cv2.COLOR_BGR2RGB)
test_img_nsy = cv2.resize(test_img_nsy, (512, 512))
test_img_nsy = test_img_nsy.astype("float32") / 255.0

fig, ax = plt.subplots(1, 3, figsize=(30,30))
ax[0].imshow(test_img_nsy)
ax[0].title.set_text('Noisy')
ax[1].imshow(pred)
ax[1].title.set_text('Pred')
ax[2].imshow(test_img_gt)
ax[2].title.set_text('GT')
plt.show()

ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv2d_5/Relu' defined at (most recent call last):
    File "c:\Software\Programs\Conda\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Software\Programs\Conda\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Software\Programs\Conda\envs\tf\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Software\Programs\Conda\envs\tf\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Software\Programs\Conda\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 737, in start
      self.io_loop.start()
    File "c:\Software\Programs\Conda\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\Software\Programs\Conda\envs\tf\lib\asynci

In [11]:
from tensorflow.keras.export import ExportArchive

# CBDNet.export('models/exported/CBDNet')

export_archive = ExportArchive()
export_archive.track(CBDNet)
export_archive.add_endpoint(
    name='serve',
    fn=CBDNet.call,
    input_signature=[tf.TensorSpec(shape=(None, 3), dtype=tf.float32)],
)

export_archive.write_out('models/exported/CBDNet')

ModuleNotFoundError: No module named 'tensorflow.keras.export'

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(CBDNet)
light_model = converter.convert()

with open('models/exported/CBDNet.tflite', 'wb') as f:
    f.write(light_model)

INFO:tensorflow:Assets written to: C:\Users\Federico\AppData\Local\Temp\tmp7ny1kgwp\assets


INFO:tensorflow:Assets written to: C:\Users\Federico\AppData\Local\Temp\tmp7ny1kgwp\assets
